In [5]:
!pip install mtcnn


In [21]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
from tensorflow.keras.models import load_model
from mtcnn import MTCNN

drowsiness_model = load_model("drowsiness_model.h5")
age_model = load_model("age_model.h5")
detector = MTCNN()

def predict_eye(img):
    try:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        gray = cv2.resize(gray, (24, 24)).reshape(1, 24, 24, 1) / 255.0
        pred = drowsiness_model.predict(gray, verbose=0)[0]
        return np.argmax(pred)  
    except:
        return 1 
        
def predict_age(face_img):
    try:
        face_img = cv2.resize(face_img, (64, 64)).reshape(1, 64, 64, 3) / 255.0
        pred = age_model.predict(face_img, verbose=0)[0][0]
        return int(pred * 100)  # Denormalize (0–1 → 0–100)
    except:
        return -1  
        
class DrowsinessApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Drowsiness Detection with Age Estimation")
        self.root.geometry("900x700")

        self.panel = tk.Label(root)
        self.panel.pack()

        btn_frame = tk.Frame(root)
        btn_frame.pack(pady=10)

        tk.Button(btn_frame, text="Select Image", command=self.load_image).grid(row=0, column=0, padx=10)
        tk.Button(btn_frame, text="Select Video", command=self.load_video).grid(row=0, column=1, padx=10)

    def process_faces(self, img):
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        faces = detector.detect_faces(img_rgb)

        sleepy_count = 0
        ages_list = []

        for face in faces:
            x, y, w, h = face['box']
            x, y = abs(x), abs(y)
            face_crop = img[y:y+h, x:x+w]

            keypoints = face.get('keypoints', {})
            lx, ly = keypoints.get('left_eye', (0, 0))
            rx, ry = keypoints.get('right_eye', (0, 0))

            # 👁️ Crop eyes for drowsiness prediction
            eye_crop_left = img[ly-12:ly+12, lx-12:lx+12] if ly and lx else None
            eye_crop_right = img[ry-12:ry+12, rx-12:rx+12] if ry and rx else None

            eye_left = predict_eye(eye_crop_left) if eye_crop_left is not None else 1
            eye_right = predict_eye(eye_crop_right) if eye_crop_right is not None else 1

            eye_state = 0 if (eye_left == 0 and eye_right == 0) else 1

            age = predict_age(face_crop)
            ages_list.append(age)

            color = (0, 0, 255) if eye_state == 0 else (0, 255, 0)
            if eye_state == 0:
                sleepy_count += 1

            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, f"Age: {age}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        return img, len(faces), sleepy_count, ages_list

    def load_image(self):
        path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg *.png *.jpeg")])
        if not path:
            return

        img = cv2.imread(path)
        result_img, total_faces, sleepy_count, ages = self.process_faces(img)

        img_disp = cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB)
        im_pil = Image.fromarray(img_disp)
        imgtk = ImageTk.PhotoImage(im_pil)

        self.panel.configure(image=imgtk)
        self.panel.image = imgtk

        msg = f"Total People: {total_faces}\nSleeping: {sleepy_count}\nAges: {ages}"
        messagebox.showinfo("Result", msg)

    def load_video(self):
        path = filedialog.askopenfilename(filetypes=[("Video files", "*.mp4 *.avi *.mov")])
        if not path:
            return

        cap = cv2.VideoCapture(path)
        sleepy_count, total_faces = 0, 0
        age_collector = []

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            processed_frame, faces_count, sleepy, ages = self.process_faces(frame)
            total_faces += faces_count
            sleepy_count += sleepy
            age_collector.extend(ages)

            cv2.imshow("Drowsiness Detection", processed_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

        msg = f"Total People Detected: {total_faces}\nSleeping: {sleepy_count}\nAges: {age_collector}"
        messagebox.showinfo("Video Summary", msg)

if __name__ == "__main__":
    root = tk.Tk()
    app = DrowsinessApp(root)
    root.mainloop()
